# Clustering Experiment 
#### By: Charbel Marche

We decided to individually tackle the problem using 1 method, and by the time we are all done we will be able to merge our techniques and select the optimal techinque. Currently we are determining only the indiviudal digits, but we need to recognize these as coherent numbers and be able to assign entries to numbers.

### Register Images to Start

To start, we need to register images using the `utilities/conversion/apply_homography_to_labels.ipynb` notebook. This should be run before running this notebook. This notebook is built on the assumption that the `data/registered_images` directory has been created and populated. Additionally it assumes that the `data/yolo_data.json` file is created. Both of these are created in the referenced notebook. 

#### Install Packages

This will be added to as I develop.

In [7]:
import os
import json
import random
from pathlib import Path
from typing import List

import cv2
import numpy as np
from PIL import Image, ImageDraw
from sklearn.cluster import KMeans

#### Start By Loading YOLO Data

To start I want to bring in the YOLO formatted data for each sheet and I can additionally load the respective images.

In [8]:
# Load yolo_data.json
PATH_TO_YOLO_DATA = '../../data/yolo_data.json'
PATH_TO_REGISTERED_IMAGES = '../../data/registered_images'
UNIFIED_IMAGE_PATH = '../../data/unified_intraoperative_preoperative_flowsheet_v1_1_front.png'
with open(PATH_TO_YOLO_DATA) as json_file:
    yolo_data = json.load(json_file)

print(f"Found {len(yolo_data)} sheets in yolo_data.json")

Found 19 sheets in yolo_data.json


Now let's select relevant bounding boxes from the blood pressure and HR zone. 

Start by defining functions to convert YOLO bounding box format to pixels (to see if the bounding box is within region of interest). Then create a function that allows you to select ROI and returns a list of bounding boxes within this ROI.

In [9]:
def YOLO_to_pixels(x_center, y_center, width, height, image_width, image_height):
    """
    Convert YOLO bounding box format to pixel coordinates

    Args:
        x_center: float, x center of the bounding box
        y_center: float, y center of the bounding box
        width: float, width of the bounding box
        height: float, height of the bounding box
        image_width: int, width of the image
        image_height: int, height of the image

    Returns:
        A single tuple with the following values:
            x_min: int, minimum x coordinate of the bounding box in pixels
            y_min: int, minimum y coordinate of the bounding box in pixels
            x_max: int, maximum x coordinate of the bounding box in pixels
            y_max: int, maximum y coordinate of the bounding box in pixels
    """
    x_min = int((float(x_center) * image_width) - (width * image_width) / 2)
    y_min = int((float(y_center) * image_height) - (height * image_height) / 2)
    x_max = int((float(x_center) * image_width) + (width * image_width) / 2)
    y_max = int((float(y_center) * image_height) + (height * image_height) / 2)
    return x_min, y_min, x_max, y_max

def select_relevant_bounding_boxes(sheet_data: List[str], path_to_image: Path) -> List[str]:
  """
  Given sheet data for bounding boxes in YOLO format, display the image and allow the user to select a region of interest (ROI).
    Identify bounding boxes that are within the selected region and draw rectangles around them. 
    Return the bounding boxes that are within the selected region.

  Args:
      sheet_data: List of bounding boxes in YOLO format.
      path_to_image: Path to the image file.

  Returns:
      Bounding boxes that are within the selected region, in YOLO format.
  """
  # Load the image
  image = cv2.imread(path_to_image)

  # Display the image and allow the user to select a ROI
  resized_image = cv2.resize(image, (800, 600))
  roi = cv2.selectROI("Select ROI", resized_image)
  print(f"ROI selected: {roi}")

  # The function returns a tuple (x, y, width, height)
  x, y, w, h = roi
  print(f"Selected region: x={x}, y={y}, w={w}, h={h}")

  # Draw a rectangle around the selected region
  cv2.rectangle(img=resized_image, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=1)

  # Close all OpenCV windows
  cv2.destroyAllWindows()

  # List of bounding boxes that are within the selected region
  bounding_boxes_within_region = []

  # Identify bounding boxes that are within the selected region
  for bounding_box in sheet_data:
    # Bounding boxes are in YOLO format, let's convert to pixels and see if they are within the selected region
    identifier, x_center, y_center, bb_width, bb_height = bounding_box.split(' ')
    x_min, y_min, x_max, y_max  = YOLO_to_pixels(float(x_center), float(y_center), float(bb_width), float(bb_height), 800, 600)
    if x_min >= x and y_min >= y and x_max <= x + w and y_max <= y + h:

      # Generate a random color for the bounding box in (0, 255, 0) scalar format
      generate_color = lambda: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

      # Draw a rectangle around the bounding box
      cv2.rectangle(img=resized_image, pt1=(x_min, y_min), pt2=(x_max, y_max), color=generate_color(), thickness=1)

      # Append the bounding box to the list of bounding boxes within the selected region
      bounding_boxes_within_region.append(bounding_box)

  # Display the image with the selected region and bounding boxes
  resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
  resized_image = Image.fromarray(resized_image)
  resized_image.show()

  return bounding_boxes_within_region


Create a function for K-means clustering

In [15]:
def cluster_kmeans(bounding_boxes: List[str], number_of_clusters: int) -> List[int]:
    """
    Cluster bounding boxes using K-Means clustering algorithm.

    Args:
        bounding_boxes: List of bounding boxes in YOLO format.
        number_of_clusters: Number of clusters to use in K-Means clustering.

    Returns:
        List of cluster labels.
    """
    # Convert to a NumPy array (using only x_center and y_center)
    data = np.array([[float(box.split(' ')[1]), float(box.split(' ')[2])] for box in bounding_boxes])

    # Apply K-Means
    kmeans = KMeans(n_clusters=number_of_clusters, init='k-means++', n_init=10, max_iter=300, tol=1e-8, random_state=42)
    kmeans.fit(data)

    # Get cluster labels
    labels = kmeans.predict(data)

    return labels

Now lets use these functions to get the relevant bounding boxes for clustering.

In [16]:
# Iterate over all images
for sheet, bounding_boxes in yolo_data.items():
    print(f"Sheet: {sheet}")
    full_image_path = os.path.join(PATH_TO_REGISTERED_IMAGES, sheet)
    print(f"Full image path: {full_image_path}")

    # Call the analyze_sheet function with data from the loop
    relevant_bounding_boxes = select_relevant_bounding_boxes(bounding_boxes, full_image_path)

    # Now we need to cluster the bounding boxes that pertain to the same multi-digit number
    labels = cluster_kmeans(relevant_bounding_boxes, 62)

    image: Image = Image.open(full_image_path)
    image_width, image_height = image.size

    label_color_map = {}
    for i, label in enumerate(labels):

        # Get the bounding box
        bounding_box = relevant_bounding_boxes[i]
        value, x_center, y_center, width, height = bounding_box.split(' ')
        x_min, y_min, x_max, y_max = YOLO_to_pixels(float(x_center), float(y_center), float(width), float(height), image_width, image_height)

        # Draw bounding boxes on the image
        generate_color = lambda: "#%06x" % random.randint(0, 0xFFFFFF)


        # If the label is not in the color map, generate a new color
        if label not in label_color_map:
            label_color_map[label] = generate_color()

        # Open the image
        draw = ImageDraw.Draw(image)

        box = [
            x_min,
            y_min,
            x_max,
            y_max,
        ]
        draw.rectangle(box, outline=label_color_map[label], width=3)

    # Display the image
    image.show()

    # Print numbers for each cluster
    for i, label in enumerate(labels):
        print(f"Cluster {label}: {relevant_bounding_boxes[i]}")

    # Break after the first image
    break

Sheet: RC_0001_intraoperative.JPG
Full image path: ../../data/registered_images\RC_0001_intraoperative.JPG
ROI selected: (104, 222, 631, 201)
Selected region: x=104, y=222, w=631, h=201
Cluster 28: 5 0.9098491136955492 0.38141891180300247 0.0047951438210227515 0.010082098268995088
Cluster 10: 2 0.13794031316583807 0.39902471804151346 0.00507585005326705 0.010464657054227944
Cluster 10: 2 0.1434342216722893 0.39900106991038603 0.005159921357125952 0.010430668849571112
Cluster 10: 0 0.14874451145981296 0.3989715576171875 0.004878808223839959 0.010312284581801445
Cluster 39: 2 0.13839600996537643 0.41458115521599265 0.005050483472419515 0.010243135340073484
Cluster 39: 1 0.14340712576201467 0.4147241689644608 0.004516906738281257 0.010044136795343106
Cluster 39: 0 0.1484095810398911 0.4144839298023897 0.004963064482717799 0.010348642386642182
Cluster 19: 2 0.13811845259232955 0.4301294424019608 0.0047374933416193254 0.010339403339460762
Cluster 19: 0 0.14345488114790483 0.4301910041360294

In [17]:
def print_classes_and_values(classes, values):
  """Prints the class number followed by all the indices and values belonging to that class.

  Args:
    classes: A list of class labels for each value.
    values: A list of values.
  """

  # Create a dictionary to store indices and values by class
  class_data = {}
  for i, class_label in enumerate(classes):
    if class_label not in class_data:
      class_data[class_label] = []
    class_data[class_label].append((i, values[i]))

  # Print the class number followed by the indices and values
  for class_label, data in class_data.items():
    print(f"Class {class_label}:")
    for index, value in data:
      print(f"  Index {index}: {value}")


relevant_numbers = [box.split(" ")[0] for box in relevant_bounding_boxes]
print_classes_and_values(labels, relevant_numbers)


Class 28:
  Index 0: 5
  Index 128: 2
Class 10:
  Index 1: 2
  Index 2: 2
  Index 3: 0
Class 39:
  Index 4: 2
  Index 5: 1
  Index 6: 0
Class 19:
  Index 7: 2
  Index 8: 0
  Index 9: 0
Class 37:
  Index 10: 1
  Index 11: 9
  Index 12: 0
Class 1:
  Index 13: 1
  Index 14: 8
  Index 15: 0
Class 29:
  Index 16: 1
  Index 17: 7
  Index 18: 0
Class 33:
  Index 19: 1
  Index 20: 6
  Index 21: 0
Class 11:
  Index 22: 1
  Index 23: 5
  Index 24: 0
Class 38:
  Index 25: 1
  Index 26: 4
  Index 27: 0
Class 21:
  Index 28: 1
  Index 29: 3
  Index 30: 0
Class 36:
  Index 31: 1
  Index 32: 2
  Index 33: 0
Class 20:
  Index 34: 1
  Index 35: 1
  Index 36: 0
Class 3:
  Index 37: 1
  Index 38: 0
  Index 39: 0
Class 40:
  Index 40: 9
  Index 41: 0
Class 46:
  Index 42: 8
  Index 43: 0
Class 17:
  Index 44: 7
  Index 45: 0
Class 44:
  Index 46: 6
  Index 47: 0
Class 25:
  Index 48: 5
  Index 49: 0
Class 42:
  Index 50: 4
  Index 51: 0
Class 7:
  Index 52: 3
  Index 53: 0
Class 60:
  Index 54: 0
Class 34